In [1]:
!pip install requests beautifulsoup4



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

base='https://markets.businessinsider.com/news/aapl-stock?p='
df=pd.DataFrame(columns=['Date','Headline'])
data=[]
for page in range (1,400):
    url=base+str(page)
    response=requests.get(url)
    soup=BeautifulSoup(response.text,'lxml')
    news= soup.find_all('div',class_='latest-news__story')
    for article in news:
        datetime = article.find('time',class_='latest-news__date').get('datetime')
        Headline = article.find('a',class_='news-link').text
        data.append([datetime,Headline])
    
df=pd.DataFrame(data,columns=df.columns)
df['date'] = pd.to_datetime(df['Date']).dt.date
df.drop(['Date'],axis='columns',inplace=True)
df1=pd.concat([df.date,df.Headline],axis='columns')
df1


KeyboardInterrupt: 

In [ ]:
df2 = df1.groupby('date')['Headline'].agg(lambda x: ' '.join(x)).reset_index()
df2

,date,Headline
0,2011-08-10,Apple uses courts to buy time to secure iPad's...
1,2011-08-12,Apple going after Google in tablet spats?
2,2011-08-16,Taiwan's HTC sues Apple over patents
3,2011-08-18,"China Mobile H1 profit grows, talking to Apple..."
4,2011-08-19,Apple moves against knockoffs in New York
...,...,...
3120,2024-07-15,"Goldilocks scenario is taking shape, brighteni..."
3121,2024-07-16,Apple Releases First Public Beta For IOS18 Th...
3122,2024-07-17,Chips lose $500B in market cap amid export con...
3123,2024-07-18,"Apple Targets Chrome, Other Browser Companies ..."


In [ ]:
df2.shape

(3125, 2)

In [ ]:
!pip install yfinance



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import yfinance as yf
import pandas as pd

# Define the stock ticker symbol
ticker = 'AAPL'

# Fetch historical stock data
stock_data = yf.download(ticker, start='2011-08-10', end='2024-07-21')

# Reset index to get 'Date' as a column
stock_data.reset_index(inplace=True)

# Print the first few rows of the stock data
stock_data


[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2011-08-10,13.255357,13.380357,12.946429,12.988929,10.981524,878656800
1,2011-08-11,13.232857,13.408929,13.025714,13.346429,11.283772,741969200
2,2011-08-12,13.502500,13.558571,13.365357,13.463929,11.383113,528976000
3,2011-08-15,13.558214,13.748929,13.503214,13.693214,11.576967,460544000
4,2011-08-16,13.625714,13.691786,13.430714,13.588571,11.488492,498750000
...,...,...,...,...,...,...,...
3251,2024-07-15,236.479996,237.229996,233.089996,234.399994,234.399994,62631300
3252,2024-07-16,235.000000,236.270004,232.330002,234.820007,234.820007,43234300
3253,2024-07-17,229.449997,231.460007,226.639999,228.880005,228.880005,57345900
3254,2024-07-18,230.279999,230.440002,222.270004,224.179993,224.179993,66034600


In [ ]:
df2.rename(columns={'date':'Date'},inplace=True)
df2['Date'] = pd.to_datetime(df2['Date'])
stock_data['Date'] = pd.to_datetime(stock_data['Date'])

# Merge DataFrames on 'Date'
df3 = pd.merge(df2, stock_data, on='Date', how='left')

In [ ]:
df3

,Date,Headline,Open,High,Low,Close,Adj Close,Volume
0,2011-08-10,Apple uses courts to buy time to secure iPad's...,13.255357,13.380357,12.946429,12.988929,10.981524,878656800.0
1,2011-08-12,Apple going after Google in tablet spats?,13.502500,13.558571,13.365357,13.463929,11.383113,528976000.0
2,2011-08-16,Taiwan's HTC sues Apple over patents,13.625714,13.691786,13.430714,13.588571,11.488492,498750000.0
3,2011-08-18,"China Mobile H1 profit grows, talking to Apple...",13.244286,13.308929,12.906071,13.073214,11.052781,851435200.0
4,2011-08-19,Apple moves against knockoffs in New York,12.934643,13.107143,12.714286,12.715357,10.750232,775888400.0
...,...,...,...,...,...,...,...,...
3120,2024-07-15,"Goldilocks scenario is taking shape, brighteni...",236.479996,237.229996,233.089996,234.399994,234.399994,62631300.0
3121,2024-07-16,Apple Releases First Public Beta For IOS18 Th...,235.000000,236.270004,232.330002,234.820007,234.820007,43234300.0
3122,2024-07-17,Chips lose $500B in market cap amid export con...,229.449997,231.460007,226.639999,228.880005,228.880005,57345900.0
3123,2024-07-18,"Apple Targets Chrome, Other Browser Companies ...",230.279999,230.440002,222.270004,224.179993,224.179993,66034600.0


In [ ]:
Label=df3.Close-df3.Open
Label

0      -0.266428
1      -0.038570
2      -0.037144
3      -0.171072
4      -0.219286
          ...   
3120   -2.080002
3121   -0.179993
3122   -0.569992
3123   -6.100006
3124   -0.510010
Length: 3125, dtype: float64

In [ ]:
df3['Label']=Label.apply(lambda x: 0 if x<=0 else 1)

In [ ]:
df3

,Date,Label,Headline,Cleaned_Headline,Open,High,Low,Close,Adj Close,Volume
0,2011-08-10,0,apple uses courts to buy time to secure ipad's...,apple us court buy time secure ipad 's market ...,13.255357,13.380357,12.946429,12.988929,10.981524,878656800.0
1,2011-08-12,0,apple going after google in tablet spats?,apple going google tablet spat,13.502500,13.558571,13.365357,13.463929,11.383113,528976000.0
2,2011-08-16,0,taiwan's htc sues apple over patents,taiwan 's htc sue apple patent,13.625714,13.691786,13.430714,13.588571,11.488492,498750000.0
3,2011-08-18,0,"china mobile h1 profit grows, talking to apple...",china mobile h1 profit grows talking apple iph...,13.244286,13.308929,12.906071,13.073214,11.052781,851435200.0
4,2011-08-19,0,apple moves against knockoffs in new york,apple move knockoff new york,12.934643,13.107143,12.714286,12.715357,10.750232,775888400.0
...,...,...,...,...,...,...,...,...,...,...
3120,2024-07-15,0,"goldilocks scenario is taking shape, brighteni...",goldilocks scenario taking shape brightening p...,236.479996,237.229996,233.089996,234.399994,234.399994,62631300.0
3121,2024-07-16,0,apple releases first public beta for ios18 th...,apple release first public beta ios18 lazarus ...,235.000000,236.270004,232.330002,234.820007,234.820007,43234300.0
3122,2024-07-17,0,chips lose $500b in market cap amid export con...,chip lose $ 500b market cap amid export contro...,229.449997,231.460007,226.639999,228.880005,228.880005,57345900.0
3123,2024-07-18,0,"apple targets chrome, other browser companies ...",apple target chrome browser company privacy pr...,230.279999,230.440002,222.270004,224.179993,224.179993,66034600.0


In [ ]:
current_columns = df3.columns.tolist()

# Find the index of the column to move
column_to_move = 'Label'
index_to_move = current_columns.index(column_to_move)

# Remove the column from its current position
current_columns.pop(index_to_move)

# Insert the column at the new index (position 0 in this case)
new_index_position = 1
current_columns.insert(new_index_position, column_to_move)

# Reorder the DataFrame columns
df3 = df3[current_columns]


In [ ]:
df3

,Date,Label,Headline,Cleaned_Headline,Open,High,Low,Close,Adj Close,Volume
0,2011-08-10,0,apple uses courts to buy time to secure ipad's...,apple us court buy time secure ipad 's market ...,13.255357,13.380357,12.946429,12.988929,10.981524,878656800.0
1,2011-08-12,0,apple going after google in tablet spats?,apple going google tablet spat,13.502500,13.558571,13.365357,13.463929,11.383113,528976000.0
2,2011-08-16,0,taiwan's htc sues apple over patents,taiwan 's htc sue apple patent,13.625714,13.691786,13.430714,13.588571,11.488492,498750000.0
3,2011-08-18,0,"china mobile h1 profit grows, talking to apple...",china mobile h1 profit grows talking apple iph...,13.244286,13.308929,12.906071,13.073214,11.052781,851435200.0
4,2011-08-19,0,apple moves against knockoffs in new york,apple move knockoff new york,12.934643,13.107143,12.714286,12.715357,10.750232,775888400.0
...,...,...,...,...,...,...,...,...,...,...
3120,2024-07-15,0,"goldilocks scenario is taking shape, brighteni...",goldilocks scenario taking shape brightening p...,236.479996,237.229996,233.089996,234.399994,234.399994,62631300.0
3121,2024-07-16,0,apple releases first public beta for ios18 th...,apple release first public beta ios18 lazarus ...,235.000000,236.270004,232.330002,234.820007,234.820007,43234300.0
3122,2024-07-17,0,chips lose $500b in market cap amid export con...,chip lose $ 500b market cap amid export contro...,229.449997,231.460007,226.639999,228.880005,228.880005,57345900.0
3123,2024-07-18,0,"apple targets chrome, other browser companies ...",apple target chrome browser company privacy pr...,230.279999,230.440002,222.270004,224.179993,224.179993,66034600.0


In [ ]:
df3.Headline=df3.Headline.str.lower()


In [ ]:
df3

,Date,Label,Headline,Cleaned_Headline,Open,High,Low,Close,Adj Close,Volume
0,2011-08-10,0,apple uses courts to buy time to secure ipad's...,apple us court buy time secure ipad 's market ...,13.255357,13.380357,12.946429,12.988929,10.981524,878656800.0
1,2011-08-12,0,apple going after google in tablet spats?,apple going google tablet spat,13.502500,13.558571,13.365357,13.463929,11.383113,528976000.0
2,2011-08-16,0,taiwan's htc sues apple over patents,taiwan 's htc sue apple patent,13.625714,13.691786,13.430714,13.588571,11.488492,498750000.0
3,2011-08-18,0,"china mobile h1 profit grows, talking to apple...",china mobile h1 profit grows talking apple iph...,13.244286,13.308929,12.906071,13.073214,11.052781,851435200.0
4,2011-08-19,0,apple moves against knockoffs in new york,apple move knockoff new york,12.934643,13.107143,12.714286,12.715357,10.750232,775888400.0
...,...,...,...,...,...,...,...,...,...,...
3120,2024-07-15,0,"goldilocks scenario is taking shape, brighteni...",goldilocks scenario taking shape brightening p...,236.479996,237.229996,233.089996,234.399994,234.399994,62631300.0
3121,2024-07-16,0,apple releases first public beta for ios18 th...,apple release first public beta ios18 lazarus ...,235.000000,236.270004,232.330002,234.820007,234.820007,43234300.0
3122,2024-07-17,0,chips lose $500b in market cap amid export con...,chip lose $ 500b market cap amid export contro...,229.449997,231.460007,226.639999,228.880005,228.880005,57345900.0
3123,2024-07-18,0,"apple targets chrome, other browser companies ...",apple target chrome browser company privacy pr...,230.279999,230.440002,222.270004,224.179993,224.179993,66034600.0


In [ ]:

import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download required nltk resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Initialize the lemmatizer and set of stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Load the English tokenizer from spaCy
nlp = spacy.load('en_core_web_sm')

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation using spaCy tokenizer
    doc = nlp(text)
    text = ' '.join([token.text for token in doc if not token.is_punct])
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stop words
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatize tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text

# Sample DataFrame with headlines


# Apply preprocessing to each headline
df3['Cleaned_Headline'] = df3['Headline'].apply(preprocess_text)




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rishi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rishi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rishi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


KeyboardInterrupt: 

In [ ]:
df3

,Date,Label,Headline,Cleaned_Headline,Open,High,Low,Close,Adj Close,Volume
0,2011-08-10,0,apple uses courts to buy time to secure ipad's...,apple us court buy time secure ipad 's market ...,13.255357,13.380357,12.946429,12.988929,10.981524,878656800.0
1,2011-08-12,0,apple going after google in tablet spats?,apple going google tablet spat,13.502500,13.558571,13.365357,13.463929,11.383113,528976000.0
2,2011-08-16,0,taiwan's htc sues apple over patents,taiwan 's htc sue apple patent,13.625714,13.691786,13.430714,13.588571,11.488492,498750000.0
3,2011-08-18,0,"china mobile h1 profit grows, talking to apple...",china mobile h1 profit grows talking apple iph...,13.244286,13.308929,12.906071,13.073214,11.052781,851435200.0
4,2011-08-19,0,apple moves against knockoffs in new york,apple move knockoff new york,12.934643,13.107143,12.714286,12.715357,10.750232,775888400.0
...,...,...,...,...,...,...,...,...,...,...
3120,2024-07-15,0,"goldilocks scenario is taking shape, brighteni...",goldilocks scenario taking shape brightening p...,236.479996,237.229996,233.089996,234.399994,234.399994,62631300.0
3121,2024-07-16,0,apple releases first public beta for ios18 th...,apple release first public beta ios18 lazarus ...,235.000000,236.270004,232.330002,234.820007,234.820007,43234300.0
3122,2024-07-17,0,chips lose $500b in market cap amid export con...,chip lose $ 500b market cap amid export contro...,229.449997,231.460007,226.639999,228.880005,228.880005,57345900.0
3123,2024-07-18,0,"apple targets chrome, other browser companies ...",apple target chrome browser company privacy pr...,230.279999,230.440002,222.270004,224.179993,224.179993,66034600.0


In [ ]:
current_columns = df3.columns.tolist()

# Find the index of the column to move
column_to_move = 'Cleaned_Headline'
index_to_move = current_columns.index(column_to_move)

# Remove the column from its current position
current_columns.pop(index_to_move)

# Insert the column at the new index (position 0 in this case)
new_index_position = 3
current_columns.insert(new_index_position, column_to_move)

# Reorder the DataFrame columns
df3 = df3[current_columns]


In [ ]:
df3

,Date,Label,Headline,Cleaned_Headline,Open,High,Low,Close,Adj Close,Volume
0,2011-08-10,0,apple uses courts to buy time to secure ipad's...,apple us court buy time secure ipad 's market ...,13.255357,13.380357,12.946429,12.988929,10.981524,878656800.0
1,2011-08-12,0,apple going after google in tablet spats?,apple going google tablet spat,13.502500,13.558571,13.365357,13.463929,11.383113,528976000.0
2,2011-08-16,0,taiwan's htc sues apple over patents,taiwan 's htc sue apple patent,13.625714,13.691786,13.430714,13.588571,11.488492,498750000.0
3,2011-08-18,0,"china mobile h1 profit grows, talking to apple...",china mobile h1 profit grows talking apple iph...,13.244286,13.308929,12.906071,13.073214,11.052781,851435200.0
4,2011-08-19,0,apple moves against knockoffs in new york,apple move knockoff new york,12.934643,13.107143,12.714286,12.715357,10.750232,775888400.0
...,...,...,...,...,...,...,...,...,...,...
3120,2024-07-15,0,"goldilocks scenario is taking shape, brighteni...",goldilocks scenario taking shape brightening p...,236.479996,237.229996,233.089996,234.399994,234.399994,62631300.0
3121,2024-07-16,0,apple releases first public beta for ios18 th...,apple release first public beta ios18 lazarus ...,235.000000,236.270004,232.330002,234.820007,234.820007,43234300.0
3122,2024-07-17,0,chips lose $500b in market cap amid export con...,chip lose $ 500b market cap amid export contro...,229.449997,231.460007,226.639999,228.880005,228.880005,57345900.0
3123,2024-07-18,0,"apple targets chrome, other browser companies ...",apple target chrome browser company privacy pr...,230.279999,230.440002,222.270004,224.179993,224.179993,66034600.0


In [ ]:
df3.Headline[3120]

'goldilocks scenario is taking shape, brightening prospects for cyclicals: bank of america apple launches new homepod mini in midnight  apple shares rise after morgan stanley endorsed apple stock as "top pick"  back-to-school spending likely to remain unchanged this year apple play store now offers first pc emulator for iphone, ipad  microsoft\'s \'iphone moment\'? wedbush analyst dan ives expects ai to add $1 trillion to valuation 3 ai stocks that could grow your wealth 3 metaverse stocks that could be multibaggers in the making: july edition british regulators to look into digital wallets of big tech dear meta stock fans, mark your calendars for july 23 solaredge layoffs 2024: what to know about the latest solar job cuts apple to rally more than 18%? here are 10 top analyst forecasts for monday morgan stanley doubles down on apple (aapl) stock with top ai call magnificent 7 showdown: one stock to buy, 2 to sell 7 billion-dollar stocks that deserve a spot in your portfolio dear taiwan

In [ ]:
df3.Cleaned_Headline[3120]

"goldilocks scenario taking shape brightening prospect cyclicals bank america apple launch new homepod mini midnight apple share rise morgan stanley endorsed apple stock top pick back school spending likely remain unchanged year apple play store offer first pc emulator iphone ipad microsoft 's iphone moment wedbush analyst dan ives expects ai add $ 1 trillion valuation 3 ai stock could grow wealth 3 metaverse stock could multibaggers making july edition british regulator look digital wallet big tech dear meta stock fan mark calendar july 23 solaredge layoff 2024 know latest solar job cut apple rally 18 10 top analyst forecast monday morgan stanley double apple aapl stock top ai call magnificent 7 showdown one stock buy 2 sell 7 billion dollar stock deserve spot portfolio dear taiwan semiconductor stock fan mark calendar july 18 nvidia stock ai giant face crossroad competition heat analyst offer insight technology company genpact g apple aapl"

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Download VADER lexicon if not already installed
nltk.download('vader_lexicon')

# Initialize the sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Function to get sentiment score
def get_sentiment_score(text):
    return sid.polarity_scores(text)['compound']

# Apply sentiment analysis
df3['Sentiment_Score'] = df3['Cleaned_Headline'].apply(get_sentiment_score)

df3


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\rishi\AppData\Roaming\nltk_data...


,Date,Label,Headline,Cleaned_Headline,Open,High,Low,Close,Adj Close,Volume,Sentiment_Score
0,2011-08-10,0,apple uses courts to buy time to secure ipad's...,apple us court buy time secure ipad 's market ...,13.255357,13.380357,12.946429,12.988929,10.981524,878656800.0,0.5574
1,2011-08-12,0,apple going after google in tablet spats?,apple going google tablet spat,13.502500,13.558571,13.365357,13.463929,11.383113,528976000.0,0.0000
2,2011-08-16,0,taiwan's htc sues apple over patents,taiwan 's htc sue apple patent,13.625714,13.691786,13.430714,13.588571,11.488492,498750000.0,0.0000
3,2011-08-18,0,"china mobile h1 profit grows, talking to apple...",china mobile h1 profit grows talking apple iph...,13.244286,13.308929,12.906071,13.073214,11.052781,851435200.0,0.4404
4,2011-08-19,0,apple moves against knockoffs in new york,apple move knockoff new york,12.934643,13.107143,12.714286,12.715357,10.750232,775888400.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
3120,2024-07-15,0,"goldilocks scenario is taking shape, brighteni...",goldilocks scenario taking shape brightening p...,236.479996,237.229996,233.089996,234.399994,234.399994,62631300.0,0.9808
3121,2024-07-16,0,apple releases first public beta for ios18 th...,apple release first public beta ios18 lazarus ...,235.000000,236.270004,232.330002,234.820007,234.820007,43234300.0,-0.3182
3122,2024-07-17,0,chips lose $500b in market cap amid export con...,chip lose $ 500b market cap amid export contro...,229.449997,231.460007,226.639999,228.880005,228.880005,57345900.0,0.8750
3123,2024-07-18,0,"apple targets chrome, other browser companies ...",apple target chrome browser company privacy pr...,230.279999,230.440002,222.270004,224.179993,224.179993,66034600.0,0.9670


In [ ]:
# Assuming 'Label' column indicates market movement (0 or 1)
correlation = df3['Sentiment_Score'].corr(df3['Label'])
correlation


-0.02791724153257411

In [ ]:
!pip install textblob

   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
    --------------------------------------- 10.2/626.3 kB ? eta -:--:--
    --------------------------------------- 10.2/626.3 kB ? eta -:--:--
    --------------------------------------- 10.2/626.3 kB ? eta -:--:--
    --------------------------------------- 10.2/626.3 kB ? eta -:--:--
    --------------------------------------- 10.2/626.3 kB ? eta -:--:--
   - -------------------------------------- 30.7/626.3 kB 87.5 kB/s eta 0:00:07
   - -------------------------------------- 30.7/626.3 kB 87.5 


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from textblob import TextBlob as tb

In [ ]:
def subjectivity(text):
    return tb(text).sentiment.subjectivity

def polarity(text):
    return tb(text).sentiment.polarity

In [ ]:
df3['Subjectivity']=df3['Cleaned_Headline'].apply(subjectivity)
df3['Polarity']=df3['Cleaned_Headline'].apply(polarity)

In [ ]:
df3

,Date,Label,Headline,Cleaned_Headline,Open,High,Low,Close,Adj Close,Volume,Sentiment_Score,Subjectivity,Polarity
0,2011-08-10,0,apple uses courts to buy time to secure ipad's...,apple us court buy time secure ipad 's market ...,13.255357,13.380357,12.946429,12.988929,10.981524,878656800.0,0.5574,0.600000,0.400000
1,2011-08-12,0,apple going after google in tablet spats?,apple going google tablet spat,13.502500,13.558571,13.365357,13.463929,11.383113,528976000.0,0.0000,0.000000,0.000000
2,2011-08-16,0,taiwan's htc sues apple over patents,taiwan 's htc sue apple patent,13.625714,13.691786,13.430714,13.588571,11.488492,498750000.0,0.0000,0.000000,0.000000
3,2011-08-18,0,"china mobile h1 profit grows, talking to apple...",china mobile h1 profit grows talking apple iph...,13.244286,13.308929,12.906071,13.073214,11.052781,851435200.0,0.4404,0.000000,0.000000
4,2011-08-19,0,apple moves against knockoffs in new york,apple move knockoff new york,12.934643,13.107143,12.714286,12.715357,10.750232,775888400.0,0.0000,0.454545,0.136364
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3120,2024-07-15,0,"goldilocks scenario is taking shape, brighteni...",goldilocks scenario taking shape brightening p...,236.479996,237.229996,233.089996,234.399994,234.399994,62631300.0,0.9808,0.449134,0.241883
3121,2024-07-16,0,apple releases first public beta for ios18 th...,apple release first public beta ios18 lazarus ...,235.000000,236.270004,232.330002,234.820007,234.820007,43234300.0,-0.3182,0.418519,0.009259
3122,2024-07-17,0,chips lose $500b in market cap amid export con...,chip lose $ 500b market cap amid export contro...,229.449997,231.460007,226.639999,228.880005,228.880005,57345900.0,0.8750,0.540000,0.560000
3123,2024-07-18,0,"apple targets chrome, other browser companies ...",apple target chrome browser company privacy pr...,230.279999,230.440002,222.270004,224.179993,224.179993,66034600.0,0.9670,0.516282,0.247760


In [ ]:
def sia(text):
    s=SentimentIntensityAnalyzer()
    sentiment=s.polarity_scores(text)
    return sentiment

compound=[]
ng=[]
pos=[]
neu=[]

SIA=0

for i in range(0,3125):
    SIA=sia(df3['Cleaned_Headline'][i])
    compound.append(SIA['compound'])
    ng.append(SIA['neg'])
    neu.append(SIA['neu'])
    pos.append(SIA['pos'])

In [ ]:
df3['Compound']=compound
df3['Positive']=pos
df3['Negative']=ng
df3['Neutral']=neu
df3.drop(['Neautral'],axis='columns',inplace=True)

In [ ]:
df3.head()
df3.to_csv('xyz.csv',index=False)